<a href="https://colab.research.google.com/github/shiz-107/shiz-107/blob/main/Harnessing_Eisenstein_Integers_and_Affine_Mapping_with_Elliptic_Curves_for_Ultra_Efficient_S_Box_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
p=257
 Parameter of EC:  b=2
 points on curve:  {Q: on EC}
 Unit point on EC: a=(0, 197) = 0+197w
 x=list of distinct y-coordinates of points on EC
 ax = [P=s+tw: list of product]
 Point addition: P+Q={u+vw:list} for all
 Sx={u; distinct list %256}=S =  [143, 42, 249, 36, 146, 139, 16, 241, 9, 176, 137, 199, 88, 208, 4, 253, 197, 117, 153, 236, 159, 195, 114, 22, 234, 135, 1, 23, 0, 244, 68, 124, 59, 113, 185, 11, 18, 239, 120, 157, 221, 92, 134, 228, 50, 187, 79, 13, 122, 98, 17, 215, 169, 61, 95, 189, 64, 136, 35, 168, 58, 8, 62, 70, 193, 178, 190, 213, 21, 29, 67, 226, 81, 104, 200, 231, 198, 158, 32, 248, 246, 52, 57, 111, 72, 44, 46, 165, 211, 255, 25, 34, 73, 144, 225, 2, 220, 204, 203, 218, 109, 20, 138, 183, 96, 97, 14, 152, 38, 242, 40, 121, 245, 103, 170, 80, 161, 83, 191, 163, 48, 219, 250, 129, 24, 173, 179, 123, 60, 91, 27, 252, 254, 162, 194, 71, 3, 188, 180, 148, 131, 171, 227, 30, 55, 182, 128, 181, 214, 149, 127, 222, 115, 141, 167, 108, 39, 100, 237, 160, 105, 86, 37, 15, 84, 112, 116, 133, 125, 102, 110, 247, 10, 7, 186, 76, 75, 140, 69, 217, 175, 107, 53, 223, 177, 184, 210, 87, 156, 106, 65, 77, 26, 145, 243, 66, 130, 12, 119, 78, 31, 126, 94, 56, 90, 118, 74, 54, 154, 33, 150, 224, 251, 5, 101, 89, 201, 93, 155, 99, 230, 172, 205, 151, 209, 43, 132, 47, 174, 49, 164, 28, 232, 51, 196, 142, 6, 166, 63, 233, 238, 216, 192, 41, 240, 202, 82, 206, 235, 45, 19, 229, 212, 207, 147, 85]
 l=104.5

THEN: Affine Mappng gives 2 S-boxes.

S1:
a=117, b=44 , l=107.25 , m=[106.0, 106.0, 108.0, 108.0, 106.0, 106.0, 110.0, 108.0]
[135, 94, 249, 160, 230, 179, 124, 81, 73, 156, 201, 31, 100, 60, 0, ...]
S2:
a=117, b=172 , l=107.25 , m=[106.0, 106.0, 108.0, 108.0, 106.0, 106.0, 110.0, 108.0]
[7, 222, 121, 32, 102, 51, 252, 209,............]

In [ ]:
######## Its full Correct Code with verified Addition, Loop on "a" taking form the "satisfying poynomials one by one",


from typing_extensions import TypeVarTuple
import sympy as sp
p = 257
coeff_combinations = [(x, y) for x in range(p) for y in range(p)]
w = sp.symbols('w')
polynomials = []
K=[]
###################### EIs
for coefficient in coeff_combinations:
    polynomial = sum(c * w**i for i, c in enumerate(coefficient))
    polynomial %= p
    polynomials.append(polynomial)

#################### point addition
def point_addition(P, Q, p):
    if P is None:
        return Q
    if Q is None:
        return P
    if P == Q:
        m = (((3 * P[0]**2) ) * pow(2 * P[1], -1, p)) % p
    else:
        if Q[0] - P[0] == 0:
            return None
        m = ((Q[1] - P[1]) * pow(Q[0] - P[0], -1, p)) % p
    x1, y1 = P
    Rx = (m**2 - x1 - Q[0]) % p
    Ry = (m * (x1 - Rx) - y1) % p
    return (Rx, Ry)

#################### computing EC
for b in range(2,3):
    count = 0
    satisfying_polynomials = []
    second_coefficient = set()

    for i, polynomial in enumerate(polynomials):
        x, y = coeff_combinations[i]
        Y = (y**2) % p
        X = (x**3 + b) % p
        if Y == X:
            satisfying_polynomials.append((x, y))  # Append the coefficients, not the index
            count += 1
            second_coefficient.add(y)

    print(satisfying_polynomials)
    print(f"b={b}, Count={count}")
    print('Distinct coefficient:')
    distinct_coeff = list(second_coefficient)
    print(distinct_coeff)

    for a in satisfying_polynomials[:2]:
        x, y = a  # Unpack the tuple into x and y
        constant_term = x
        coefficient_of_w = y
        R = []
        ######################### computing ax for all x in dist y coordinates on EC
        for x in distinct_coeff:
            result_prime = x * constant_term % p
            result_dou = x * coefficient_of_w % p
            R.append((result_prime, result_dou))

        print("Value of 'a':", a)  # Print the value of 'a'
        print("R", R)
        y_coor_list = []
        resulttt = []

        P = None  ###########Initialize P as None
        for i in range(len(R)):
            P = R[i]  ############### looop for P as the current point in R=ax
            for Q_coefficients in satisfying_polynomials:
                Q = Q_coefficients  ############# loop for the use of coefficients of satisfyng polynmial as Q
                #print(f"P{i + 1}:", P)
                #print(f"Q:", Q)
                result = point_addition(P, Q, p)
                #print(f"P{i + 1} + Q:", result)  ################### P+Q
                resulttt.append(result)
        print("list of point_addition=", resulttt)
        Sx = []
        Sy = []
        dist = set()
        distt = set()
        for P in resulttt:
            if P == None:  ##################### Excluding None element
                continue
            u = int(P[0] % 256)
            v = int(P[1] % 256)
            if u not in dist:
                dist.add(u)
                Sx.append(u)
            if v not in distt:
                distt.add(v)
                Sy.append(v)
        print("Sx=", Sx)
        print(len(Sx))
        #print("Sy=", Sy)
        #print(len(Sy))
        K.append((Sx,Sy,a,b))
#print(K)

for i, string in enumerate(K):
      A =string[0]
      b=[]
      for i in range (len(A)):
          if A[i] not in b:
              b.append(A[i])
      print(len(b))
      #print(S)#148\106.125
      ###############################
      import numpy as np
      a = np.array([[1,1,1,1],[1,-1,1,-1],[1,1,-1,-1],[1,-1,-1,1]])
      b = np.concatenate((a,a),1)
      c = np.concatenate((a,-a),1)
      d = np.concatenate((b,c),0)
      #print(d)
      b1 = np.concatenate((d,d),1)
      c1 = np.concatenate((d,-d),1)
      d1 = np.concatenate((b1,c1),0)
      #print(d1)
      b2 = np.concatenate((d1,d1),1)
      c2 = np.concatenate((d1,-d1),1)
      d2 = np.concatenate((b2,c2),0)
      #print(d2)
      b3 = np.concatenate((d2,d2),1)
      c3 = np.concatenate((d2,-d2),1)
      d3 = np.concatenate((b3,c3),0)
      #print(d3)
      b4 = np.concatenate((d3,d3),1)
      c4 = np.concatenate((d3,-d3),1)
      d4 = np.concatenate((b4,c4),0)
      #print(d4)
      b5 = np.concatenate((d4,d4),1)
      c5 = np.concatenate((d4,-d4),1)
      d5 = np.concatenate((b5,c5),0)
      #print(d5)
      ####################################
      B = []
      for i in range (8):
          B.append([])
          for j in range (len(A)):
              k = A[j]//(2**i)
              B[i].append(k)
      #print(B)
      A = []
      for i in range (8):
          A.append([])
          for j in range (len(B[i])):
              k = (-1)**(B[i][j])
              A[i].append(k)
      #print(A)
      ################
      C = []
      D = []
      for i in range (8):
          D.append([])
          k = d5.dot(A[i])
          C.append(k)
          for j in range (len(C[i])):
              k = abs(C[i][j])
              D[i].append(k)
      h = []
      g = []
      for i in range (8):
          g.append(max(D[i]))
          k = 128-(g[i]/2)
          h.append(k)
      print(h)
      l = sum(h)/len(h)
      print('Non-linearity',l)
      print(f"a={string[2]} , b={string[3]} , l={l}")

In [ ]:
from typing_extensions import TypeVarTuple
import sympy as sp

p = 257
coeff_combinations = [(x, y) for x in range(p) for y in range(p)]
w = sp.symbols('w')
polynomials = []
K=[]

###################### EIs
for coefficient in coeff_combinations:
    polynomial = sum(c * w**i for i, c in enumerate(coefficient))
    polynomial %= p
    polynomials.append(polynomial)

#################### point addition
def point_addition(P, Q, p):
    if P is None:
        return Q
    if Q is None:
        return P
    if P == Q:
        m = (((3 * P[0]**2) ) * pow(2 * P[1], -1, p)) % p
    else:
        if Q[0] - P[0] == 0:
            return None
        m = ((Q[1] - P[1]) * pow(Q[0] - P[0], -1, p)) % p
    x1, y1 = P
    Rx = (m**2 - x1 - Q[0]) % p
    Ry = (m * (x1 - Rx) - y1) % p
    return (Rx, Ry)
b = 2
count = 0
satisfying_polynomials = []
second_coefficient = set()

for i, polynomial in enumerate(polynomials):
    x, y = coeff_combinations[i]
    Y = (y**2) % p
    X = (x**3 + b) % p
    if Y == X:
        satisfying_polynomials.append((x, y))  # Append the coefficients, not the index
        count += 1
        second_coefficient.add(y)

print(satisfying_polynomials)
print(f"b={b}, Count={count}")
print('Distinct coefficient:')
distinct_coeff = list(second_coefficient)
print(distinct_coeff)

for a in satisfying_polynomials[:2]:
    x, y = a  # Unpack the tuple into x and y
    constant_term = x
    coefficient_of_w = y
    R = []
    ######################### computing ax for all x in dist y coordinates on EC
    for x in distinct_coeff:
        result_prime = x * constant_term % p
        result_dou = x * coefficient_of_w % p
        R.append((result_prime, result_dou))

    print("Value of 'a':", a)  # Print the value of 'a'
    print("R", R)
    y_coor_list = []
    resulttt = []

    P = None  ###########Initialize P as None
    for i in range(len(R)):
        P = R[i]  ############### looop for P as the current point in R=ax
        for Q_coefficients in satisfying_polynomials:
            Q = Q_coefficients  ############# loop for the use of coefficients of satisfyng polynmial as Q
            print(f"P{i + 1}:", P)
            print(f"Q:", Q)
            result = point_addition(P, Q, p)
            print(f"P{i + 1} + Q:", result)  ################### P+Q
            resulttt.append(result)
    print("list of point_addition=", resulttt)
    Sx = []
    dist = set()
    for P in resulttt:
        if P == None:  ##################### Excluding None element
            continue
        u = int(P[0] % 256)
        if u not in dist:
            dist.add(u)
            Sx.append(u)
    print("Sx=", Sx)
    print(len(Sx))


In [ ]:
from math import gcd              #a=(0, 197) , b=2 , l=104.5
import numpy as np

S =  [143, 42, 249, 36, 146, 139, 16, 241, 9, 176, 137, 199, 88, 208, 4, 253, 197, 117, 153, 236, 159, 195, 114, 22, 234, 135, 1, 23, 0, 244, 68, 124, 59, 113, 185, 11, 18, 239, 120, 157, 221, 92, 134, 228, 50, 187, 79, 13, 122, 98, 17, 215, 169, 61, 95, 189, 64, 136, 35, 168, 58, 8, 62, 70, 193, 178, 190, 213, 21, 29, 67, 226, 81, 104, 200, 231, 198, 158, 32, 248, 246, 52, 57, 111, 72, 44, 46, 165, 211, 255, 25, 34, 73, 144, 225, 2, 220, 204, 203, 218, 109, 20, 138, 183, 96, 97, 14, 152, 38, 242, 40, 121, 245, 103, 170, 80, 161, 83, 191, 163, 48, 219, 250, 129, 24, 173, 179, 123, 60, 91, 27, 252, 254, 162, 194, 71, 3, 188, 180, 148, 131, 171, 227, 30, 55, 182, 128, 181, 214, 149, 127, 222, 115, 141, 167, 108, 39, 100, 237, 160, 105, 86, 37, 15, 84, 112, 116, 133, 125, 102, 110, 247, 10, 7, 186, 76, 75, 140, 69, 217, 175, 107, 53, 223, 177, 184, 210, 87, 156, 106, 65, 77, 26, 145, 243, 66, 130, 12, 119, 78, 31, 126, 94, 56, 90, 118, 74, 54, 154, 33, 150, 224, 251, 5, 101, 89, 201, 93, 155, 99, 230, 172, 205, 151, 209, 43, 132, 47, 174, 49, 164, 28, 232, 51, 196, 142, 6, 166, 63, 233, 238, 216, 192, 41, 240, 202, 82, 206, 235, 45, 19, 229, 212, 207, 147, 85]

K=[]
p=257
for a in range(117,118):
    for b in range(1,p):
              R = []
              distinct_s = set()
              for z in S:
                        s =int(a*z+b)%256
                        if s not in distinct_s:
                            distinct_s.add(s)
                            R.append(s)
              if len(R) == 256:
                    #print(R)
                    K.append((a, b ,R))


for i, result in enumerate(K):
    A = result[2]
    b = []
    for i in range(len(A)):
        if A[i] not in b:
            b.append(A[i])
    a = np.array([[1, 1, 1, 1], [1, -1, 1, -1], [1, 1, -1, -1], [1, -1, -1, 1]])
    b = np.concatenate((a, a), 1)
    c = np.concatenate((a, -a), 1)
    d = np.concatenate((b, c), 0)

    b1 = np.concatenate((d, d), 1)
    c1 = np.concatenate((d, -d), 1)
    d1 = np.concatenate((b1, c1), 0)

    b2 = np.concatenate((d1, d1), 1)
    c2 = np.concatenate((d1, -d1), 1)
    d2 = np.concatenate((b2, c2), 0)

    b3 = np.concatenate((d2, d2), 1)
    c3 = np.concatenate((d2, -d2), 1)
    d3 = np.concatenate((b3, c3), 0)

    b4 = np.concatenate((d3, d3), 1)
    c4 = np.concatenate((d3, -d3), 1)
    d4 = np.concatenate((b4, c4), 0)

    b5 = np.concatenate((d4, d4), 1)
    c5 = np.concatenate((d4, -d4), 1)
    d5 = np.concatenate((b5, c5), 0)

    B = []
    for i in range(8):
        B.append([])
        for j in range(len(A)):
            k = A[j] // (2 ** i)
            B[i].append(k)

    A = []
    for i in range(8):
        A.append([])
        for j in range(len(B[i])):
            k = (-1) ** (B[i][j])
            A[i].append(k)

    C = []
    D = []
    for i in range(8):
        D.append([])
        k = d5.dot(A[i])
        C.append(k)
        for j in range(len(C[i])):
            k = abs(C[i][j])
            D[i].append(k)
    h = []
    g = []
    for i in range(8):
        g.append(max(D[i]))
        k = 128 - (g[i] / 2)
        h.append(k)
    l = sum(h) / len(h)

    if l >= 107:
        print(f"a={result[0]}, b={result[1]} , l={l} , m={h}")
        print(result[2])

a=117, b=44 , l=107.25 , m=[106.0, 106.0, 108.0, 108.0, 106.0, 106.0, 110.0, 108.0]
[135, 94, 249, 160, 230, 179, 124, 81, 73, 156, 201, 31, 100, 60, 0, 205, 53, 165, 25, 8, 215, 75, 70, 58, 30, 223, 161, 175, 44, 176, 64, 216, 35, 209, 185, 51, 102, 103, 4, 237, 45, 56, 106, 96, 6, 163, 71, 29, 238, 246, 241, 111, 105, 13, 151, 141, 108, 84, 43, 244, 174, 212, 130, 42, 97, 134, 2, 133, 197, 109, 203, 118, 49, 180, 148, 191, 170, 98, 204, 132, 154, 240, 57, 231, 20, 72, 50, 149, 155, 183, 153, 182, 137, 252, 1, 22, 184, 104, 243, 206, 253, 80, 62, 207, 12, 129, 146, 164, 138, 198, 116, 121, 37, 63, 222, 188, 193, 27, 119, 171, 28, 67, 110, 33, 36, 61, 251, 99, 152, 195, 131, 88, 66, 54, 214, 159, 139, 24, 112, 208, 11, 83, 235, 226, 79, 90, 172, 229, 250, 69, 55, 162, 187, 157, 127, 136, 255, 224, 125, 76, 41, 122, 21, 7, 144, 92, 48, 245, 77, 202, 114, 15, 190, 95, 46, 232, 115, 40, 181, 89, 39, 19, 101, 23, 17, 68, 38, 239, 120, 158, 225, 93, 14, 113, 59, 86, 150, 168, 143, 210, 87, 